# **Administração de Sistemas**

## **Configurações servidor NFS e NIS**

## **Instalação de Máquinas**

Versão minimal CentOs

## **Configurações da VM**

* Disco -> 10gb
* Memória RAM -> 1gb
* Mudar placa de rede para **"Bridged Adapter"**

## **Configurações do CentOS**

* Mudar linguagem
* Mudar esquema do teclado
* Desativar o Kdump
* Na parte de network
    * Mudar o hostname
    * Ativar a network
* Desativar a *Security Policy*
* Partições
    * *I will configure partitioning*
    * *Standard Partition*
        * **"/boot"** -> 250m
        * **"swap"** -> 2g
        * **"/"** -> deixar em branco
        * Mudar **"/boot"** e **"/"** para **ext4** 
* Colocar password de root

## **Primeiros Passos**

### **Servidor e Cliente**

#### **Desativar a firewall**

* **Comandos**
    * systemctl stop firewalld
    * systemctl disable firewalld
    * vi /etc/selinux/config
        * Remover a parte onde diz **"SELINUX=enforcing"** para **"SELINUX=disabled"**
    * reboot   



* **Instalar Network Tools**
    * **Comando**
        * yum install net-tools -y

## **Criação de users e grupos**

#### **Criação de user**

* **Comandos**
    * useradd nomeuser
    * passwd nomeuser (definir pass para o user)

#### **Criação de grupos**

* **Comandos**
    * Criar um grupo
        * groupadd nomegrupo
    * Adicionar users ao grupo
        * gpasswd -a nomeuser nomegrupo
    * Definir administrador do grupo
        * gpasswd -A nomeuser nomegrupo
    * Remover users do grupo (apenas admin do grupo)
        * gpasswd -d nomeuser nomegrupo
    * Diretoria comum para o grupo
        * mkdir /home/nomepastagrupo
        * chgrp nomegrupo /home/nomepastagrupo
        * chmod g+ws /home/nomepastagrupo

## **DHCP Server**

* yum install dchp -y
* vi /etc/dhcp/dhcpd.conf
    * Colocar estas configurações, ips tem de ser conforme pedido
        * subnet 192.168.0.0 network 255.255.255.0 { <br />
            range 192.168.0.100 192.1698.0.200;

            default-lease-time 86400; <br />
            max-lease-time 86400;

            option routers 192.168.0.254;

            option broadcast-address 192.168.0.255; <br />
            option subnet-mask 255.255.255.0;

            option domain-name-servers 192.168.0.10,192.168.0.11;

            option domain-name "estig.pt";

            host nomePC.estig.pt { <br />
                hardware ethernet MACMaqClient;<br />
                fixed-address colocarIPPretendido; <br />
            } <br />
        } <br />
        ddns-update-style none;
* systemctl start dhcpd
* systemctl enable dhcpd

## **Preparação NFS e NIS**

##### Instalação dos serviços de *NFS* e *NIS*

### **Servidor e Cliente**

* **Comandos**
    * yum install nfs-utils -y

### **Servidor**

* **Comandos**
    * systemctl start nfs
    * systemctl enable nfs
    * yum install ypserv -y

### **Cliente**

* **Comandos**
    * yum install ypbind -y

* **Mudar placa para Internal Network**

## **Configuração do NFS**

### **Opções**

* rw -> Acesso de leitura e escrita
* ro -> Acesso de leitura
* async -> Escreve os dados quando o servidor sente que precisa
* sync -> Escreve os dados quando são recebidos

### **Servidor e Cliente**

* Definir IP
    * **Comandos**
        * nmtui
            * Mudar IPv4 para *Manual*
                * Colocar IP dado pelo prof
        * systemctl restart network

### **Servidor**

* Criação da pasta pretendida
    * **Comandos**
        * mkdir /pasta
        * chmod 777 /pasta
        * vi /etc/exports
            * **Colocar o texto**
            * /pasta -tab-  192.168.50.0/24(rw,hide,sync)
        * systemctl restart nfs

### **Cliente**

* Criação da pasta partilhada (Nomes das pastas podem ser diferentes)
    * **Comandos**
        * mkdir /pasta
        * chmod 777 /pasta/
        * mount -t nfs 192.168.50.1:/pasta /pasta (ip do servidor)

## **Configuração do NIS**

### **Servidor**

* **Comandos**
    * systemctl start ypserv
    * systemctl enable ypserv
    * nisdomainname estig.pt (nome que é pedido)
    * cd /var/yp
    * make
    * vi /etc/exports
        * **Colocar o texto**
            * /home -tab- 192.168.50.0/24(rw,hide,sync)
    * systemctl restart nfs

### **Cliente**

* **Comandos**
    * authconfig-tui
        * Selecionar a opção "use NIS" (usar o espaço)
            * domain -> estig.pt (nome colocado no domain name no servidor)
            * server -> 192.168.50.1 (ip do servidor)
    * systemctl enable ypbind
    * systemctl start ypbind
    * mount -t nfs 192.168.50.1:/home /home
    * vi /etc/fstab
        * Colocar o texto após os discos
            * 192.168.50.1:/home /home nfs rw 0 0

## **Quotas**

### **Servidor**

* Comandos
    * yum install quota quota-devel -y
    * vi /etc/fstab
        * Colocar defaults,grpquota,usrquota na partição "/"
    * reboot
    * mount -o remount /
    * quotacheck -cugm /
    * edquota -u user
        * Colocar a memória pretendida em *"blocks" "soft" e "hard"*
    * quotaon /


## **Servidor FTP**

### **Servidor**

* **Comandos**
    * yum install vsftpd -y
    * systemctl start vsftpd
    * systemctl enable vsftpd

* **Ficheiros**
    * vi /etc/vsftpd/vsftpd.conf -> Ficheiro com as configurações do ftp

* **Enjaular users**
    * vi /etc/vsftpd/vsftpd.conf
        * Descomentar:
	        * chroot_local_user=YES
	        * chroot_list_enable=YES
        * Colocar:
            * chroot_list_file=/etc/vsftpd_ftp_list
            * allow_writeable_chroot=YES
    * vi /etc/vsftpd/ftp_list
    * chmod 755 ftp_list
    * systemctl restart vsftpd

* **Adicionar exceção a um user**
    * vi /etc/vsftpd/ftp_list
        * Colocar nome user

## **Notas**

Se forem criados um ou mais novos users no servidor tem que se repetir os comandos para que possam ser mapeadas as pastas desses users.:
* cd /var/yp
* make


### **Comandos úteis**

* du -h * -> ver espaço utilizado pelo user
* df -h * -> ver o espaço utilizado por cada file da pasta

#### **Links**

Para criar um symbolic link para um ficheiro ou pasta.
* **Comandos**
    * *Symbolic Link*
        * ln -s origem destino

#### **Permissões**

* Comando chmod ???
* Tipos de permissões
    * 000 -> sem permissão
    * 001 -> permissão de execução
    * 010 -> permissão de escrita
    * 011 -> permissão de execução e escrita
    * 100 -> permissão de leitura
    * 101 -> permissão de execução e leitura
    * 110 -> permissão de escrita e leitura
    * 111 -> todas as permissões
